In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


download_path = '/Users/jonathanoh/Desktop/down'
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# script_dir = "/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/crawling"
# download_path = "/Users/jonathanoh/Desktop/contents/exiftoexcel/temp_image_zip_treated"
# s = Service('/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/chromedriver-mac-arm64/chromedriver')





def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기
    i = 0

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                i+=1
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                title = title.replace("/", "_")
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))


        
            
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
#         if i==0:
#             return True
#         else :
#             return False  # 오류 발생시 실패로 간주

    if len(failed_downloads)==0 or i == 0 :
        return True
    else : 
        return False  
    



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    
    duplicate_url = combine_csv_files_remove_duplicates(script_dir)
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urlsx = duplicate_url.loc[:, 'url'].to_list()
    old_urls = []
    for x in old_urlsx :
        old_urls.append(x.split('?')[0])

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href.split('?')[0] in old_urls : 
                mm = href.split('?')[0].split('/')[-1]
                print(f"Processing: {total} 중 {i} {mm}__{title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href.split('?')[0], title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [2]:
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'


options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)

driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')

In [3]:


# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

[('https://arca.live/b/aiartreal/95408394?p=1', '한국 시장거리 컨셉 사진', 'APPLE_'),
 ('https://arca.live/b/aiartreal/95408393?p=1',
  '밤거리 (hanimix_real_v16A)',
  '너예쁘다'),
 ('https://arca.live/b/aiartreal/95408151?p=1', '새해 인사.', '크런키z'),
 ('https://arca.live/b/aiartreal/95407838?p=1',
  'hanimix_v19(HQ3D) 막뽑은 여고생짤',
  'Taizo'),
 ('https://arca.live/b/aiartreal/95407414?p=1', '내년 목표는 독서', '냥이집사'),
 ('https://arca.live/b/aiartreal/95407019?p=1', '파판7 리메이크 티파 실사버전', '달의영혼'),
 ('https://arca.live/b/aiartreal/95406864?p=1', 'sdxl 테스트용 와카 샘플그림', '리스펙트'),
 ('https://arca.live/b/aiartreal/95406785?p=1', '한해를 보내며 올려보는 란제리짤', 'SLIME'),
 ('https://arca.live/b/aiartreal/95406718?p=1',
  '그래도 게임하는거보단 Ai하는게 낫지',
  'kkoor'),
 ('https://arca.live/b/aiartreal/95405593?p=1',
  '(스압주의)올해의 애착모델 TPix_typeUR_v0.2_fp16',
  '냥이집사'),
 ('https://arca.live/b/aiartreal/95404944?p=1',
  '올해 한 해 고생 많으셨습니다..한복 // hanimix_XL_v12+BMAB+SDXL_Hanbok_LoRA_beta',
  '호릉불'),
 ('https://arca.live/b/aiartreal/95404397?p=1', '한복 판타지',

In [3]:
post_list = []
for i in range(3):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)

post_list.reverse()
post_list

[('https://arca.live/b/aiartreal/95382489?p=3', '자러가기전 업로드', '비스비스'),
 ('https://arca.live/b/aiartreal/95382790?p=3',
  'from behind, bent over, all fours',
  '저기있는곰'),
 ('https://arca.live/b/aiartreal/95382997?p=3',
  'hanimix_real_v16A 부먹',
  '빠다파리'),
 ('https://arca.live/b/aiartreal/95383665?p=3',
  '가방 든 여인 (해상도 조절 실패 6144*6144)',
  'PromptShaper'),
 ('https://arca.live/b/aiartreal/95384596?p=3', '이 넘은 뭐하는 넘일까요?', '산체스'),
 ('https://arca.live/b/aiartreal/95385519?p=3', 'hanimix_real_v16A 부먹', '포르투'),
 ('https://arca.live/b/aiartreal/95385672?p=3',
  'hanimix_v19(HQ_3D) / 찍먹',
  '북이'),
 ('https://arca.live/b/aiartreal/95385706?p=3',
  'hanimix_real_v16B - 그림을 그리랬더니 예술을 하고 있네.. 찢었다..',
  '빠다파리'),
 ('https://arca.live/b/aiartreal/95386400?p=3', '디즈니의 흙역사 수정', '몬TDAI'),
 ('https://arca.live/b/aiartreal/95386750?p=3',
  'Flamix Roco V1, V2, V3 and 2.5D 4개 모델 공유합니다.',
  'baiblaibiei'),
 ('https://arca.live/b/aiartreal/95386781?p=3',
  'hanimix_v19(HQ_3D)찍먹] 빛의 수호자들',
  '멍무이드아'),
 ('https

In [4]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : 자러가기전 업로드
중복 : from behind, bent over, all fours
중복 : hanimix_real_v16A 부먹
중복 : 가방 든 여인 (해상도 조절 실패 6144*6144)
중복 : 이 넘은 뭐하는 넘일까요?
중복 : hanimix_real_v16A 부먹
중복 : hanimix_v19(HQ_3D) / 찍먹
중복 : hanimix_real_v16B - 그림을 그리랬더니 예술을 하고 있네.. 찢었다..
중복 : 디즈니의 흙역사 수정
중복 : Flamix Roco V1, V2, V3 and 2.5D 4개 모델 공유합니다.
중복 : hanimix_v19(HQ_3D)찍먹] 빛의 수호자들
중복 : How to Learn From Failures
중복 : (스압) 연말 시상식
중복 : 치어리더
중복 : hanimix_XL_v12 추가 테스트샷
중복 : orgasm, hetero, spread legs,
중복 : superhentai ,mermaid lora 찍먹] 세일러비너스 머메이드~
중복 : 2024년은 공유/다운로드 중단 합니다. (Feat. 새해 복 많이 받으세요)
중복 : 새해복 많이 받으세요~
중복 : 어쩌다 8K
중복 : super-hentai 찍먹
중복 : hanimix_real_v16 찍먹
중복 : 모두 새해에는 하시는일 잘되시길(영상하나투척)
중복 : hanimix v19(HQ_3D) 찍먹!
중복 : 빨간망토 (hanimix_v19(HQ_3D))
중복 : 드디어 이노프리님의 모델을 써보았습니다
중복 : 빙) 사탕
중복 : hanimix_XL_v11Ac + comfyui
중복 : 님들 마법사옷을 뽑고싶은데 이런 옷을 어케할까요
중복 : 패트리온 노출이 조금씩 일어나기 시작하네요?
중복 : i2i 배치 돌리면서 prompt를 각 이미지마다 다르게 적용하는 방법이 있을까요?
중복 : 열심히 일해요/열심히 놀아요
중복 : 새해 복 많이 받으세요
중복 : 눈오는 동탄의 경계선 (NewHorizon)
중복 : 드러머
중복 : hani

failed : https://arca.live/b/aiartreal/95440914?p=1
Processing: 133 중 124 95441213__NeMix MuSS-T v1.25.0 미리보기 찍먹  _  thec
Processing: 133 중 125 95442518__??? : 챈붕쿤! 오늘 일출 보러 가기로 했잖아!?  _  Taizo
failed : https://arca.live/b/aiartreal/95442518?p=1
Processing: 133 중 126 95443066__해피 뉴 이어!!  _  마타나
Processing: 133 중 127 95444099__주위에서 흔하게 볼 수 있는 K-여군(저격수)  _  달의영혼
Processing: 133 중 128 95444608__서술형 랜덤와카 1  _  빠다파리
Processing: 133 중 129 95444779__풍등  _  빠다파리
failed : https://arca.live/b/aiartreal/95444779?p=1
Processing: 133 중 130 95444980__서술형 랜덤와카 3  _  빠다파리
Processing: 133 중 131 95446537__Flamix Roco V2&V3 찍먹  _  adoniel
failed : https://arca.live/b/aiartreal/95446537?p=1
Processing: 133 중 132 95446827__SDXL 알록달록 외계 행성  _  thec
Processing: 133 중 133 95447969__hanimix_real_v16B 테스트  _  달의영혼
_____finished______
Failed posts:
User: 냥이집사 Title: 여자와 자동차 컨셉( hash: 8e9d589ce9, Model: t3), URL: https://arca.live/b/aiartreal/95412505?p=2
User: Taizo Title: hanimix_XL_v12 막뽑은 여고생짤, URL: https://a

In [ ]:
list = process_images(driver, list, download_path)

Processing: 10 중 1 95417213__2023년을 추억하며.. 새해 인사 드립니다  _  하니타카
failed : https://arca.live/b/aiartreal/95417213?p=2
Processing: 10 중 2 95419608__럭셔리 수영장 1:1 매니져....(스압)  _  실사좋아
failed : https://arca.live/b/aiartreal/95419608?p=2
Processing: 10 중 3 95419623__새해에는 게임도 좀 해야지  _  정배통프살
failed : https://arca.live/b/aiartreal/95419623?p=2
Processing: 10 중 4 95420542__2024년 새해 복 많이 받으세요.  _  빠다파리
failed : https://arca.live/b/aiartreal/95420542?p=2
Processing: 10 중 5 95422307__hanimix_v19(HQ_3D) 레트로 그외 여러짤..  _  젬마
failed : https://arca.live/b/aiartreal/95422307?p=2
Processing: 10 중 6 95426799__손이 자꾸 사타구니를 가리는데  _  소앵
failed : https://arca.live/b/aiartreal/95426799?p=1
Processing: 10 중 7 95428751__super hentai 모델 어디서 공유함??  _  소앵


In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [12]:
temp = []
for url, title, user in list :
    temp.append([url.split('?')[0], title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'
csv_fail_fiile = os.path.join(script_dir, 'fail_list.csv')
df.to_csv(csv_fail_fiile, index=False)
df = combine_csv_files_remove_duplicates(script_dir)

In [9]:
df = combine_csv_files_remove_duplicates(script_dir)

In [10]:
df

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4931442.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4924345.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926329.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926315.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926328.jpeg
...,...,...
2212,https://arca.live/b/aiartreal/95302106,쉿! 그녀가 자고있어요
2213,https://arca.live/b/aiartreal/95302742,아르테미스 리얼 2.1 갖고 계신분
2214,https://arca.live/b/aiartreal/95306617,듀얼 그래픽카드 세팅 해보신 분 계실까요?
2215,https://arca.live/b/aiartreal/95307865,계정을 삭제하면 안 되는 이유


In [11]:
k = [list[5]]
del list[5]

In [52]:
df['url'] = df['url'].apply(lambda x: x.split('?')[0])

In [53]:
df.to_csv(os.path.join(script_dir, "url_datas.csv"), index=False)

<bound method NDFrame.head of                                                     url  \
0     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
1     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
2     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
3     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
4     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
...                                                 ...   
7226             https://arca.live/b/aiartreal/95066061   
7227             https://arca.live/b/aiartreal/95053202   
7228             https://arca.live/b/aiartreal/95052445   
7229             https://arca.live/b/aiartreal/95074503   
9718             https://arca.live/b/aiartreal/95074892   

                                               filename  
0                              civitai_Year_127497.jpeg  
1                              civitai_Year_175719.jpeg  
2                              civitai_Year_295008.jpeg  
3                            